# Laboratorium 12 Równania różniczkowe i zagadnienie początkowe
### Autor: Krzysztof Hardek

In [139]:
import numpy as np

## Zadanie 1 Metoda Rungego-Kutty 
Funkcja pomocnicza

In [140]:
def rung_kut4(f, x0, y0, h, n):
    xn = x0
    yn = y0
    
    for i in range(n):     
        k1 = h * f(xn, yn)
        k2 = h * f(xn + h/2, yn + k1/2)
        k3 = h * f(xn + h/2, yn + k2/2)
        k4 = h * f(xn + h, yn + k3)
        
        d_yn = (k1 + 2 * k2 + 2 * k3 + k4) / 6

        yn += d_yn
        xn += h
    
    return xn, yn

### Zalety metody w porównaniu do metody z szeregami Taylora
Metoda Rungego-Kutty ma tą zaletę w porównaniu do metody z szeregami Taylora, że nie musimy ręcznie wyliczać kolejnych pochodnych funkcji.

### Rozwiązanie zagadnienia początkowego danego równaniem $ x' = x/t + tsec(x/t) $ z warunkiem początkowym $ x(0) = 0 $
Ustawiam liczbę kroków na $ 2^7 $ tak aby skończyć na 1. Nie mogę jako warunku początkowego ustawić (0, 0), ponieważ nie jest w dziedzinie funkcji dwóch zmiennych będącej po prawej stronie równania

In [141]:
def f(t, x):
    return x/t + t * (1 / np.cos(x / t))


x, y = rung_kut4(f, 0.000000001, 0, 2 ** (-7), 2 ** 7)
print(x, y)

1.000000001 1.5191251415534004


Porównanie z dokładnym wynikiem ($ \pi/2 $)

In [142]:
print(abs(np.pi/2 - y))

0.051671185241496165


#### Wnioski
Biorąc pod uwagę prostote implementacji oraz problemy związane z wartością początkową, metoda zwraca dobre przybliżenie wyniku

### Zagadnienie początkowe $ x' = 100(\sin(t) - x) $ z warunkiem początkowym $ x(0) = 0 $

In [143]:
def g(t, x):
    return 100 * (np.sin(t) - x)

h = 0.015

In [144]:
print(rung_kut4(g, 0, 0, 0.015, int(3/0.015)))

(3.0000000000000027, 0.15100302946455946)


h = 0.02

In [145]:
print(rung_kut4(g, 0, 0, 0.02, int(3/0.02)))

(3.000000000000002, 0.15099613280114094)


h = 0.025

In [146]:
print(rung_kut4(g, 0, 0, 0.025, int(3/0.025)))

(2.9999999999999933, 0.150943166101132)


h = 0.03

In [147]:
print(rung_kut4(g, 0, 0, 0.03, int(3/0.03)))

(2.999999999999995, 672890582787.5087)


#### Wnioski
Metoda Rungego-Kutty ma to do siebie, że dla szybko zmieniających się funkcji (badana funkcja zdecydowanie taka jest) krok całkujący musi być mały, żeby błąd się nie powiększał. W zadanych przykładach 0.015, 0.02 oraz 0.025 są jeszcze odpowiednie i dają dobre przybliżenie wyniku, ale 0.3 jest już zdecydowanie za duże. Przy zwiększaniu kroku błąd będzie jeszcze większy. Generalnie przy implementacji tego algorytmu stosuje się dodatkowe monitorowanie błędu i ewentualnie robi się poprawki wraz z wyliczaniem rozwiązania.

## Zadanie 2 Adaptacyjna metoda Rungego-Kutty-Fehlberga
Funkcje pomocnicze

In [148]:
def rk45(f, x0, y0, h):
    c20 = 0.25
    c21 = 0.25
    c30 = 0.375
    c31 = 0.09375
    c32 = 0.28125
    c40 = 12/13
    c41 = 1932/2197
    c42 = -7200/2197
    c43 = 7296/2197
    c51 = 439/216
    c52 = -8.
    c53 = 3680/513
    c54 = -845/4104
    c60 = 0.5
    c61 = -8/27
    c62 = 2.
    c63 = -3544/2565
    c64 = 1859/4104
    c65 = -0.275
    a1 = 25/216
    a2 = 0
    a3 = 1408/2565
    a4 = 2197/4104
    a5 = -0.2
    b1 = 16/135
    b2 = 0
    b3 = 6656/12825
    b4 = 28561/56430
    b5 = -0.18
    b6 = 2/55
    
    k1 = h * f(x0, y0)
    k2 = h * f(x0 + c20*h, y0 + c21*k1)
    k3 = h * f(x0 + c30*h, y0 + c31*k1 + c32*k2)
    k4 = h * f(x0 + c40*h, y0 + c41*k1 + c42*k2 + c43*k3)
    k5 = h * f(x0 + h, y0 + c51*k1 + c52*k2 + c53*k3 + c54*k4)
    k6 = h * f(x0 + c60*h, y0 + c61*k1 + c62*k2 + c63*k3 + c64*k4 + c65*k5)
    
    y4 = y0 + a1*k1 + a3*k3 + a4*k4 + a5*k5
    
    y0 += b1*k1 + b3*k3 + b4*k4 + b5*k5 + b6*k6
    x0 += + h
    
    e = abs(y0 - y4)
    
    return x0, y0, e


def rk45_adaptive(f, x0, y0, h, xb, itmax, e_max, e_min, h_min, h_max):
    iflag = 1
    delta = 0.5 * (10 ** -5)
    d = -1
    y_save = -1
    x_save = -1
    k = 0
    
    while k <= itmax:
        k += 1
        
        if abs(h) < h_min:
            h = np.sign(h) * h_min
        
        if abs(h) > h_max:
            h = np.sign(h) * h_max
        
        d = abs(xb - x0)
        
        if d <= abs(h):
            iflag = 0
            if d <= delta * max([abs(xb), abs(x)]):
                break
            h = np.sign(h) * d
        
        y_save = y0
        x_save = x0
        
        x0, y0, e = rk45(f, x0, y0, h)
        
        if iflag == 0:
            break
        
        if e < e_min:
            h = 2*h
        
        if e > e_max:
            h = h/2
            y = y_save
            x = x_save
            k = k - 1
    
    return x0, y0, iflag

Test z Kincaid i Cheney

In [149]:
def func(x, y):
    return 3 + 5 * np.sin(x) + 0.2 * y

rk45_adaptive(func, 0, 0, 0.01, 10.0, 1000, 10 ** -5, 10 ** -8, 10 ** -6, 1)

(10.0, 135.9172466935473, 0)

### Rozwiązanie zagadnienia początkowego $ x' = 3x/t + 9/2t - 13 $ z warunkiem brzegowym $ x(3) = 6 $

In [150]:
def fi(t, x):
    return 3 * (x/t) + t * 4.5 - 13

def solution(t):
    return t ** 3 - (t ** 2) * 4.5 + 6.5 * t

print(rk45_adaptive(fi, 3, 6, -0.01, 0.5, 1000, 9 * (10 ** -9), 10 ** -9, 10 ** -8, 1))
print(solution(0.5))

(0.5, 2.2500000062784338, 0)
2.25


### Wnioski
Wyniki są bardzo zbliżone do siebie (9 miejsc po przecinku). Można to osiągnąc za pomocą standardowej impmelentacji: zamiast kroku h wykonać dwa kroki h/2 i porównać różnice i na jej podstawie zmniejszać bądź zwiększać krok. Jest to jednak zbyt czasochłone. Zamiast tego liczymy RK rzędu piątego i róznica rk4 i rk5 jest naszym błędem. Mając tą informacje możemy odpowiednio manipulować krokiem, tak aby zarówno jego wartość jaki wartość błedu spełniała z góry narzucone limity. Otrzymujemy w ten sposób rozwiązanie w miarę szybkie oraz dokładne.